In [1]:
# adapted from
# https://github.com/mrsac7/CSES-Solution-Extractor

In [2]:
from bs4 import BeautifulSoup
import requests
import urllib.request 
import os
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [3]:
def getQuestions():
    ques = dict()
    r = session.get(r'https://cses.fi/problemset/').content
    soup = BeautifulSoup(r, 'html.parser')
    for t in soup.find_all('li', class_ = 'task'):
        quesID = t.a['href'].split('/')[-1]
        ques[quesID] =  t.a.string
    return ques

In [4]:
def createSession(username, password):
    loginData = {
        'nick': username,
        'pass': password
    }
    with requests.Session() as s:
        url = 'https://cses.fi/login'
        soup = BeautifulSoup(s.get(url).content, 'html.parser')
        loginData['csrf_token'] = soup.find('input', attrs = {'name':'csrf_token'})['value']
        r = s.post(url, data = loginData)
        return s



In [5]:
def getSolution(ques, sol):
    r = session.get(sol).content
    soup = BeautifulSoup(r, 'html.parser')
    code = soup.find('pre', class_ = "linenums").get_text()
    return code,soup

In [6]:
def findCorrectSolution(ques):
    r = session.get(r"https://cses.fi/problemset/view/"+ques+"/").content
    soup = BeautifulSoup(r,'html.parser')
    if int(soup.find('p').string.split()[-1]) == 0: # No of submissions
        return None

    for link in soup.find_all('a', attrs = {'class':'details-link'}):
        sol = link['href']
        res = soup.find('a', href = sol, class_ ='').span['class'][2] #get status of a solution
        if res == 'full':
            return 'https://cses.fi'+sol
    return None


In [7]:
r = requests.get("https://cses.fi/problemset/").content
soup = BeautifulSoup(r,'html.parser')
username = input('username: ')
password = input('passowrd: ')
session = createSession(username, password)
questions = getQuestions()
for _ in questions:
    sol = findCorrectSolution(_);
    if not sol:
        continue
    code,urls = getSolution(_, sol)
    
    
    counter=3
    for a in urls.select("div.samp-actions"):
        c=a.select("a.save")
        url=c[0].get("href")
        k=url.split("/")
        k[-3]
        folder(f"./{_}-{questions[_]}/{counter//3}/")
        filename=f"./{_}-{questions[_]}/{counter//3}/{k[-3]}.txt"
        url="https://cses.fi/"+url
        urllib.request.urlretrieve(url, filename)
        counter+=1
    
        path = f"./{_}-{questions[_]}/The_Code.py"
        file = open(path, 'w')
        file.writelines(code.split('\n'))
        file.close()
